## Practical session 2. Nucleotide substitution & MCMC


In [29]:
import os
cwd = os.getcwd()
print(cwd)

import numpy as np # numpy is the standard numeric libray
import scipy.stats as ss # scipy is the scientific library, the stats module contains different functions
import matplotlib.pyplot as plt # matplotlib is the standard plotting library


/home/sebastian-dohne/Documents/Bayesiansstats/slides/notebooks


In [30]:
import seaborn as sns # library for statistical plotting
import pymc as pm # library for Bayesian MCMC inference
import arviz as az # library to analyze and plot Bayesian results
import pandas as pd


In this practical we want to infer different substitution rates by comparing two different DNA sequences. The data that we will be using is the 12S rRNA alignment of human and orangutan. The output of the alignment is the count of different nucleotides which consists of 948 base pairs (Genbank accesion numbers for the human and orangutan sequences are D38112 and NC_001646, respectively (Horai et al. (1995)).):

In [31]:
data_df = pd.read_csv('orangutan_human.csv',index_col = 0)
display(data_df)

length = np.sum(data_df['A_human']+data_df['C_human']+data_df['G_human']+data_df['T_human'])

N_TC = data_df.loc['C_orangutan']['T_human'] + data_df.loc['T_orangutan']['C_human'] # transitions_TC
N_AG = data_df.loc['G_orangutan']['A_human'] + data_df.loc['A_orangutan']['G_human'] # transitions_AG

N_TA = data_df.loc['T_orangutan']['A_human'] + data_df.loc['A_orangutan']['T_human'] # transversion_TA
N_TG = data_df.loc['T_orangutan']['G_human'] + data_df.loc['T_orangutan']['G_human'] # transversion_TG
N_CG = data_df.loc['G_orangutan']['C_human'] + data_df.loc['C_orangutan']['G_human'] # transversion_CG
N_CA = data_df.loc['C_orangutan']['A_human'] + data_df.loc['A_orangutan']['C_human'] # transversion_CG


N_transition = N_TC+N_AG
N_transversion = N_TA+N_CG+N_CG+N_CA
N_intact = length - N_transition - N_transversion

print("Sequence length = ", length)
print("Number of transitionss = ",N_transition)
print("Number of transversions = ",N_transversion)

FileNotFoundError: [Errno 2] No such file or directory: 'orangutan_human.csv'

There are in total 90 base differences between both mitochondrial 12s RNAs. From this substitutions we have 84 transitions (i.e. ($T\!\iff\!C$) and ($A\!\iff\!G$)), and 6 transversions (rest of substitutions). In Kimura’s 1980 (K80) nucleotide substitution model that has two parameters:

1) The sequence distance $d$ between species. $d=rt$ is the product of the average mutation rate $f$ and the branch length $t$. Since the time difference is usually unknown (unless there are fossil calibrations), one canot differentiate between a fast mutation rate over a long time, agains a slow mutation over a long time. On the contrary the distance $d=rt$ controlling the number of mutations that occured can be inferred when comparing two sequences.

 2) The transition/transversion rate ratio ($\kappa$ = rate_transition/rate_transversion) which accounts for the fact that transitions often occur at higher rates than transversions ($\kappa\gg 1$).

We need to write a function that returns us the log-likelihood given the distance between two sequences ($d$) and the ratio ($\kappa$), which can written as $P(data|d,\kappa)$. The process is a Markovian, and following similar arguments to the ones we saw on the probability theory session we know the probability distribution for each site after a time t is:

- Probability of a site staying the same:
$P_{intact} = \frac{1}{4} + \frac{1}{4}e^{-4kd/(2k+1)} + \frac{1}{2}e^{-2dk}$
- Probability of a site showing a transition:
$P_{transition} = \frac{1}{4} + \frac{1}{4}e^{-4kd/(2k+1)} - \frac{1}{2}e^{-2dk}$
- Probability of a site showing a transversion:
$P_{transversion} = \frac{1}{2} - \frac{1}{2}e^{-4kd/(2k+1)}$

The resulting likelihood for our observation is the multinomial distribution (similar to the binomial but with more than 2 possible types of results):

In [ ]:
def P_intact(k,d):
    return 0.25 + 0.25*np.exp(-4*k*d/(2*k+1)) + 0.5*np.exp(-2*d*k)
def P_transition(k,d):
    return 0.25 + 0.25*np.exp(-4*k*d/(2*k+1)) - 0.5*np.exp(-2*d*k)
def P_transversion(k,d):
    return 0.5 - 0.5*np.exp(-4*k*d/(2*k+1))

def log_likelihood(k,d): # this corresponds to a multinomial, check it!
    return N_intact*np.log(P_intact(k,d))+N_transition*np.log(P_transition(k,d))+N_transversion*np.log(P_transversion(k,d)) 
# usually is easier numerically to work with log_likelihood than likelihoods since they can be really small
    

**Q1.1** Check that the probabilities of finding a nucleotide intact or mutating make sense by plotting the three probability components $P_j(k)$ for different values of $d$, and exploring the limit cases of $d$, $k$. 

**Q1.2 Brute force inference** If we want to obtain the posterior distribution $P(k,d|data)$ we need to evaluate the distributions at *all* possible values of $k$ and $d$. Since this is impossible we could also run MCMC to sample $P(k,d|data)$ in a smart way. A more rudimentary alternative is to divide the parameter space (k,d) into a grid and evaluate the posterior for those values.

In [ ]:

# First you need to find a limit for your prior that make sense, it should be hard to find them by eye,
# but you can evaluate the likelihood at different values of k,d to have an idea of the ranges
min_k = #
max_k = #
min_d = #
max_d = #

k_range = np.linspace(min_k,max_k,100)
d_range = np.linspace(min_d,max_d,100)
K,D = np.meshgrid(k_range,d_range)

posterior = np.exp(log_likelihood(K,D)) # I could multiply by a prior as well


plt.contourf(k_range, d_range, posterior, levels=50, cmap='viridis')  # Contour plot



**Q1.3 MCMC**. We can also infer the parameters running an implemented MCMC. Implement the MCMC using the loglikelihood defined in the previous plot

In [ ]:
with pm.Model() as Pea_Model:
    k_prior = ### Define priors for k and d
    d_prior = ###
    
    # we need to define our custom likelihood
    def log_likelihood_name(name,k,d):
        return log_likelihood(k,d)
    
    f_likelihood = ### Check the documentation from pymc on how to introduce a custom likelihood
    mcmc_sample = pm.sample(4000, chains = 2, return_inferencedata = True)  

In [ ]:
az.summary(mcmc_sample) # summary of MCMC statistics
posterior = mcmc_sample.to_dataframe()
sns.kdeplot(posterior,x=('posterior', 'k'),y=('posterior', 'd'),
            cmap=plt.cm.viridis,fill=True,thresh=0,levels = 100)


**Q1.4 Just for the braves** Code an MCMC yourself. You can follow the steps of the algorithm we discussed in class. You will have freedom on the choices of the proposal step (take into account the different lengthscales in k and d). Make sure to analyze the trace to evaluate if your results has converged. 